# Processing time series

This notebook runs through some sanity checks about the photon flux calculations, comparing them to Bijan's spreadsheet results. This is to ensure that we properly process the input cubes, and gives an idea of how everything is done. Here we take the fiducial example of 47 Uma c.

In [1]:
%pylab inline --no-import-all
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
#plt.rc('text', usetex=True)

import seaborn as sns
sns.set_style("whitegrid")
matplotlib.rcParams['axes.labelsize'] = 18
import sys
import os

folder = '../../../../crispy'
if folder not in sys.path: sys.path.append(folder)

from crispy.params import Params
folder = '../../../crispy'
par = Params(folder)



Populating the interactive namespace from numpy and matplotlib


## Calculate photon fluxes

### Calculate star rates

In [2]:
import glob

# load first filelist to get its shape
fileshape = (45,150,150)


import astropy.units as u
import astropy.constants as c
import astropy.analytic_functions as af

planet_radius = 1.27*c.R_jup
ref_star_T=9600#9377*u.K
ref_star_Vmag=2.37
target_star_T=5887*u.K
target_star_Vmag=5.03

lamc=660.
BW=0.18
R=50
n_ref_star_imgs=30
Nlam = 45
lamlist = lamc*np.linspace(1.-BW/2.,1.+BW/2.,Nlam)*u.nm
npixperdlam = 2
PSFLetWidth = 2

tel_pupil_area=np.pi*(2.37/2.*u.m)**2*0.835

from crispy.tools.inputScene import convert_krist_cube,calc_contrast_Bijan
target_star_cube = convert_krist_cube(fileshape,lamlist,target_star_T,target_star_Vmag,tel_pupil_area)
ref_star_cube = convert_krist_cube(fileshape,lamlist,ref_star_T,ref_star_Vmag,tel_pupil_area)
print("Telescope area: {:.4E}".format(tel_pupil_area))
print("Photons from the target star over entire band: Us: %.3e; Bijan: %.3e" % (np.mean(target_star_cube[:,0,0]/45*0.18*lamc),1.14e8*tel_pupil_area.value))
print("Photons from the reference star over entire band: Us: %.3e; Bijan: %.3e" % (np.mean(ref_star_cube[:,0,0]/45*0.18*lamc),1.07e9*tel_pupil_area.value))
print("Planet contrast at maximum of the spectrum: {:.4E}".format(np.amax(calc_contrast_Bijan(lamlist))))
print("Planet flux within band: Us: {:.4E}".format(np.sum(target_star_cube[:,0,0]/45*0.18*lamc/45.*calc_contrast_Bijan(lamlist)))+"; Bijan: {:.4E}".format(0.885*tel_pupil_area.value))
print('This makes sense since we use a real spectrum, that contains absorption lines and such')
print("Planet flux within single spectral channel (lossless case): {:.4f}".format(np.sum(target_star_cube[:,0,0]/45*0.18*lamc/45.*calc_contrast_Bijan(lamlist))/(npixperdlam*BW*R))+" assuming %d spectral channels"% (npixperdlam*BW*R))
print("Planet flux within single pixel (lossless case): {:.4f}".format(np.sum(target_star_cube[:,0,0]/45*0.18*lamc/45.*calc_contrast_Bijan(lamlist))/(npixperdlam*BW*R*PSFLetWidth))+" assuming %d pixels per spectral channel"% (PSFLetWidth))


Telescope area: 3.6836E+00 m2
Photons from the target star over entire band: Us: 4.710e+08; Bijan: 4.199e+08
Photons from the reference star over entire band: Us: 4.245e+09; Bijan: 3.941e+09
Planet contrast at maximum of the spectrum: 7.9584E-09
Planet flux within band: Us: 3.5690E+00; Bijan: 3.2600E+00
This makes sense since we use a real spectrum, that contains absorption lines and such
Planet flux within single spectral channel (lossless case): 0.1983 assuming 18 spectral channels
Planet flux within single pixel (lossless case): 0.0991 assuming 2 pixels per spectral channel


### Test zodi calculations

In [3]:
from crispy.tools.inputScene import zodi_cube
local_zodi_mag = 23
exozodi_mag = 22
D = 2.37
pixarea = (0.1*770*1e-9/D/4.848e-6)**2
dist=14.1
absmag = target_star_Vmag-5*np.log10(dist/10.)

zodicube = zodi_cube(target_star_cube,
                     area_per_pixel=pixarea,
                     absmag=absmag,
                     Vstarmag = target_star_Vmag,
                     zodi_surfmag=local_zodi_mag,
                     exozodi_surfmag=exozodi_mag,
                     distAU=3.6,t_zodi=0.09)
print "Total ph/sec/as2/m2",np.mean(np.sum(np.sum(zodicube,axis=2),axis=1),axis=0)*0.18*lamc/tel_pupil_area
print "Total ph/sec/pix/m2",np.mean(np.sum(np.sum(zodicube,axis=2),axis=1),axis=0)*0.18*lamc/tel_pupil_area/150**2
print "Total ph/sec/lenslets/m2",np.mean(np.sum(np.sum(zodicube,axis=2),axis=1),axis=0)*0.18*lamc/tel_pupil_area/150**2*25
print "Total ph/sec/lenslet",np.mean(np.sum(np.sum(zodicube,axis=2),axis=1),axis=0)*0.18*lamc/150**2*25
print "Total ph/sec/lenslet/nm",np.mean(np.sum(np.sum(zodicube,axis=2),axis=1),axis=0)/150**2*25
print "Total ph/sec/lenslet/channel",np.mean(np.sum(np.sum(zodicube,axis=2),axis=1),axis=0)*0.18*lamc/150**2*25/9
print "Total ph/fr/lenslet/channel",np.mean(np.sum(np.sum(zodicube,axis=2),axis=1),axis=0)*0.18*lamc/150**2*25*100/9
print "Total ph/fr/ifspix/channel",np.mean(np.sum(np.sum(zodicube,axis=2),axis=1),axis=0)*0.18*lamc/150**2*25*100/9/4


Total ph/sec/as2/m2 0.996076061813 1 / m2
Total ph/sec/pix/m2 4.42700471917e-05 1 / m2
Total ph/sec/lenslets/m2 0.00110675117979 1 / m2
Total ph/sec/lenslet 0.00407683413341
Total ph/sec/lenslet/nm 3.43167856347e-05
Total ph/sec/lenslet/channel 0.000452981570379
Total ph/fr/lenslet/channel 0.0452981570379
Total ph/fr/ifspix/channel 0.0113245392595


# Check photometry at detector

In [4]:
import astropy.units as u


In [5]:
from crispy.tools.postprocessing import process_planet
tel_pupil_area=np.pi*(2.37/2.*u.m)**2*0.835
offaxis_psf_filename='/Users/mrizzo/IFS/OS5/offaxis/spc_offaxis_psf.fits'
from astropy.io import fits
par.savePoly=True
detector = process_planet(par,offaxis_psf_filename=offaxis_psf_filename,
                fileshape=fits.open(offaxis_psf_filename)[0].data.shape,
                lamlist=lamlist,
                lamc=lamc,
                outdir_average=par.exportDir,
                planet_radius = 1.27*c.R_jup,
                planet_AU = 3.6,planet_dist_pc=14.1,
                target_star_T=5887*u.K, target_star_Vmag=5.03,
                tel_pupil_area=3.650265060424805*u.m**2)


crispy - INFO - Recentering off-axis cube
crispy - INFO - Read data from HDU 0 of /Users/mrizzo/IFS/OS5/offaxis/spc_offaxis_psf.fits
crispy - INFO - The number of input pixels per lenslet is 5.000000
crispy - INFO - Writing data to ../../../crispy/SimResults/offaxiscube.fits
crispy - INFO - Constructing off-axis cube at planet separation: 4.44 lam/D (0.26 arcsec, 8.89 lenslets)
crispy - INFO - Writing data to ../../../crispy/SimResults/offaxiscube_shifted.fits
crispy - INFO - Writing data to ../../../crispy/SimResults/offaxiscube_processed.fits
crispy - INFO - The number of input pixels per lenslet is 5.000000
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths


Process Consumer-6:
Traceback (most recent call last):
Process Consumer-1:
Process Consumer-3:
Process Consumer-7:
Process Consumer-4:
Process Consumer-8:
Traceback (most recent call last):
Traceback (most recent call last):
Process Consumer-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/mrizzo/anaco

KeyboardInterrupt: 

In [ ]:
from crispy.tools.image import Image
from crispy.tools.detector import readDetector
from crispy.IFS import reduceIFSMap
par.nonoise=True
det = readDetector(par,Image(par.exportDir+'/offaxis_planet.fits'),inttime=1)
outkey = fits.HDUList(fits.PrimaryHDU(det))
outkey.writeto(par.exportDir+'/detector_offaxis.fits',clobber=True)
reduceIFSMap(par,par.exportDir+'/detector_offaxis.fits')

In [ ]:
print("Sum of of flux before detector w/o losses (ph-electrons per second):",np.sum(detector))
print("Sum of detector flux w/ losses (electrons per second):",np.sum(det))
print("Sum of flux at lenslet array (ph per second):",np.nansum(Image(par.exportDir+'/detector_offaxis_red_optext.fits').data))

flux_cube = np.sum(target_star_cube[:,0,0]/45*0.18*lamc/45.*calc_contrast_Bijan(lamlist))
print("Flux in input cube:",flux_cube)
print("Throughput of offaxis PSF:",np.sum(fits.open(offaxis_psf_filename)[0].data))
print("Throughput of shifted offaxis PSF:",np.nansum(fits.open(par.exportDir+'/offaxiscube_shifted.fits')[1].data))
from scipy.interpolate import interp1d
loadQE = np.loadtxt(par.codeRoot+"/"+par.QE)
QEinterp = interp1d(loadQE[:,0],loadQE[:,1])
QEvals = QEinterp(lamlist)
print("QE:",np.mean(QEvals))
print("Total estimated number of counts:",flux_cube*np.sum(fits.open(offaxis_psf_filename)[0].data)*np.mean(QEvals))
print("Total estimated number of counts #2:",np.sum(target_star_cube[:,0,0]/45*0.18*lamc*calc_contrast_Bijan(lamlist)*np.sum(np.sum(fits.open(offaxis_psf_filename)[0].data,axis=2),axis=1)*QEvals))

In [ ]:
from crispy.tools.image import Image
poly = Image(par.exportDir+'/detectorFramepoly.fits')
numslice = 25
print(np.sum(poly.data[numslice]))
print(target_star_cube[numslice,0,0]/45*0.18*lamc*calc_contrast_Bijan(lamlist)[numslice]*np.sum(fits.open(offaxis_psf_filename)[0].data[numslice])*QEvals[numslice])
print("Percentage error in slice:",(np.sum(poly.data[numslice])-target_star_cube[numslice,0,0]/45*0.18*lamc*calc_contrast_Bijan(lamlist)[numslice]*np.sum(fits.open(offaxis_psf_filename)[0].data[numslice])*QEvals[numslice])/(np.sum(poly.data[numslice])))


In [ ]:
print np.mean(target_star_cube[:,0,0]/45*0.18*770*10**(0.4*target_star_Vmag))/tel_pupil_area
print(1/np.sqrt(pixarea))
print(np.sum(np.sum(zodicube,axis=2),axis=1))
print np.mean(target_star_cube[:,0,0]/45*0.18*770*10**(0.4*target_star_Vmag-0.4*(local_zodi_mag)))/tel_pupil_area

# Test reduction with flatfield

In [ ]:
from crispy.unitTests import testCreateFlatfield
from crispy.tools.image import Image
offaxis_psf_filename='/Users/mrizzo/IFS/OS5/offaxis/spc_offaxis_psf.fits'
from crispy.tools.reduction import calculateWaveList
shape = Image(offaxis_psf_filename).data.shape
lam_midpts,lam_endpts = calculateWaveList(par,Nspec=shape[0],method='optext')
print lam_midpts
dlam = lam_endpts[1]-lam_endpts[0]
print dlam
par.savePoly=True
par.saveRotatedInput=True
testCreateFlatfield(par,pixsize=0.1,npix = shape[1],Nspec=shape[0],pixval = 1./dlam,useQE=True)

In [ ]:
from crispy.IFS import reduceIFSMap
reduceIFSMap(par,par.unitTestsOutputs+'/flatfield.fits')


In [ ]:
print(np.nansum(Image(par.unitTestsOutputs+'/flatfield.fits').data))
print(np.nansum(Image(par.exportDir+'/flatfield_red_optext.fits').data))


# Star photometry

In [ ]:
from crispy.tools.postprocessing import processReferenceCubes,processTargetCubes
filename = '/Users/mrizzo/IFS/OS5/with_lowfc/os5_spc_031.fits'
# processReferenceCubes(par,xshift=0.0,yshift=0.0,order=3,
#                 outdir_time_series = par.exportDir,
#                 ref_input_list=[filename],
#                 process_cubes=True,
#                 ref_star_T=9377*u.K, ref_star_Vmag=2.37,
#                 lamc=660.,BW = 0.18,
#                 tel_pupil_area=3.650265060424805*u.m**2)
processTargetCubes(par,[filename],
                outdir_time_series = par.exportDir,
                process_cubes=True,
                target_star_T=5887*u.K, target_star_Vmag=5.03,
                lamc=660.,BW = 0.18,
                tel_pupil_area=3.650265060424805*u.m**2,
                )


In [ ]:
from crispy.tools.image import Image
from crispy.tools.detector import readDetector
from crispy.IFS import reduceIFSMap
par.nonoise=True
det = readDetector(par,Image(par.exportDir+'/os5_spc_031_targetstar_IFS.fits'),inttime=1)
outkey = fits.HDUList(fits.PrimaryHDU(det))
outkey.writeto(par.exportDir+'/os5_spc_031_targetstar_IFS_detector.fits',clobber=True)
reduceIFSMap(par,par.exportDir+'/os5_spc_031_targetstar_IFS_detector.fits')

### Compare directly to Bijan's fluxtable

In [ ]:
Bij_lams = [0.36,
0.44,
0.55,
0.70,
0.80,
0.90,
1.05,
1.26,
1.64,
2.18,
3.40,
5.00,
10.20,
21.00]
Bij_flux_densities = [4.35e-08,
7.20e-08,
3.92e-08,
1.76e-08,
1.21e-08,
8.30e-09,
5.72e-09,
3.40e-09,
1.18e-09,
3.90e-10,
7.30e-11,
2.12e-11,
1.23e-12,
6.80e-14]
plt.figure(figsize=(10,10))
plt.loglog(Bij_lams,Bij_flux_densities,label="Bijan's zero-points")
BB6 = af.blackbody_lambda(Bij_lams*u.um, 6000*u.K).to(u.Watt/u.m**2/u.um/u.sr)
BB9 = af.blackbody_lambda(Bij_lams*u.um, 9000*u.K).to(u.Watt/u.m**2/u.um/u.sr)

plt.loglog(Bij_lams,BB6/BB6[2]*Bij_flux_densities[2],label='6000K star blackbody')
plt.loglog(Bij_lams,BB9/BB9[2]*Bij_flux_densities[2],label='9000K star blackbody')
plt.xlabel('Wavelength (um)')
plt.ylabel(r'Flux density zero-point (W/m2/um)')#($Wm^{-2}\mu m^{-1}$)')
plt.title('All curves normalized to same Vband flux density',fontsize = 22)
plt.legend(fontsize=18)


In [ ]:
plt.figure(figsize=(10,10))
Bij_energies = c.c*c.h/Bij_lams
plt.loglog(Bij_lams,Bij_flux_densities/Bij_energies,label="Bijan's zero-points")
BB6 = af.blackbody_lambda(Bij_lams*u.um, 6000*u.K).to(u.Watt/u.m**2/u.um/u.sr)
BB9 = af.blackbody_lambda(Bij_lams*u.um, 9000*u.K).to(u.Watt/u.m**2/u.um/u.sr)

plt.loglog(Bij_lams,BB6/BB6[2]*Bij_flux_densities[2]/Bij_energies,label='6000K star blackbody')
plt.loglog(Bij_lams,BB9/BB9[2]*Bij_flux_densities[2]/Bij_energies,label='9000K star blackbody')
plt.xlabel('Wavelength (um)')
plt.ylabel(r'Photon rate zero-point (photons/s/m2/um)')#($\textrm{photons} s^{-1} m^{-2}\mu m^{-1}$)')
plt.title('Photon rates',fontsize = 22)
plt.legend(fontsize=18)


In [ ]:
plt.figure(figsize=(10,10))
Bij_energies = c.c*c.h/Bij_lams
BB6 = af.blackbody_lambda(Bij_lams*u.um, 6000*u.K).to(u.Watt/u.m**2/u.um/u.sr)
plt.semilogx(Bij_lams,BB6/BB6[2]*Bij_flux_densities[2]/Bij_flux_densities,label="Ratio of photon rate")

plt.xlabel('Wavelength (um)')
plt.ylabel(r'Ratio of photon rates')
plt.title('Ratio of photon rates (Blackbody over Bijan) assuming Vband equivalence',fontsize = 22)
plt.legend(fontsize=18)


In [ ]:
from scipy.interpolate import interp1d
ratios =interp1d(Bij_lams,BB6/BB6[2]*Bij_flux_densities[2]/Bij_flux_densities)
print("Ratio at 0.7 microns (our flux over what Bijan finds):")
print(ratios(0.7))

# Comparison to John Krist's files

The purpose of this section is to examine the files from John Krist to see if everything checks out compared to Bijan's estimates

In [ ]:
from crispy.tools.image import Image
from astropy.io import fits
from crispy.IFS import propagateIFS
from crispy.params import Params
codefolder = '../../../crispy'
par = Params(codefolder)
print(par.exportDir)
par.saveDetector = False

psf_time_series_folder='/local/data/nicolaus2/mrizzo/haystacks/for_gsfc/with_lowfc'

# load the filenames
filelist = glob.glob(psf_time_series_folder+'/*')
filelist.sort()

# load first filelist to get its shape
fileshape = Image(filename=filelist[0]).data.shape

tel_pupil_area = 4.412*u.m**2*0.835

# reference and target star cube conversions
ref_star_cube = convert_krist_cube(fileshape,lamlist,ref_star_T,ref_star_Vmag,tel_pupil_area)
target_star_cube = convert_krist_cube(fileshape,lamlist,target_star_T,target_star_Vmag,tel_pupil_area)


# Pick one of the images in the time series
reffile = filelist[45]
cube = fits.open(reffile)[0]
print("Sum of John Krist's cube: %e" %np.sum(cube.data))
cube.data*=target_star_cube
#cube.write('test_Krist_cube.fits',clobber=True)
out = fits.HDUList(fits.PrimaryHDU(cube.data.astype(np.float32)))
out.writeto(par.exportDir + '/test_Krist_cube.fits', clobber=True)
print("Total counts from star at primary after obscuration: %e" % np.sum(target_star_cube[:,0,0]))
print("Sum of cube in photons per second: %f (residual speckle rate for entire area w/o focal plane mask)" % np.sum(cube.data))
print("Coronagraph contrast (no focal plane mask): %e " % (np.sum(cube.data)/np.sum(target_star_cube[:,0,0])))

In [ ]:
offaxispsf= '/local/data/nicolaus2/mrizzo/haystacks/for_gsfc/spc_offaxis_psf.fits'
offaxis = fits.open(offaxispsf)[0]
print("Sum of John Krist's offaxis: %f" %np.sum(offaxis.data))
fileshape=offaxis.data.shape
target_star_cube = convert_krist_cube(fileshape,lamlist,target_star_T,target_star_Vmag,tel_pupil_area)

contrast = calc_contrast_Bijan(lamlist)
contrast_cube = np.zeros(offaxis.data.shape)
for i in range(offaxis.data.shape[0]):
    contrast_cube[i,:,:] += contrast[i]*offaxis.data.shape[0]


offaxis.data*=target_star_cube*contrast_cube
out = fits.HDUList(fits.PrimaryHDU(offaxis.data.astype(np.float32)))
out.writeto(par.exportDir + '/test_Krist_offaxis.fits', clobber=True)
print("Total counts per second from planet at primary after obscuration: %e" % np.sum(target_star_cube[:,0,0]*contrast))
print("Sum of cube in photons per second: %f" % np.sum(offaxis.data))
print("Planet raw throughput (no reflections/transmissions): %e " % (np.sum(offaxis.data)/np.sum(target_star_cube[:,0,0]*contrast)))
plt.plot(lamlist,target_star_cube[:,0,0]*contrast)

### Process an image to see the detector map

In [ ]:
par.saveRotatedInput = True
from crispy.IFS import propagateIFS

if offaxis.header['LAM_C']==0.8:
    if lamc==770.:
        offaxis.header['LAM_C']=0.77
        # by NOT changing the pixelsize, we implicitly assume that the PSF is the same at 770 then at 800
    else:
        offaxis.header['LAM_C']=lamc/1000.
        offaxis.header['PIXSIZE']*=lamc/0.77
else:
    offaxis.header['LAM_C']=lamc/1000.
    offaxis.header['PIXSIZE']*=lamc/0.77
par.saveDetector=True
pol=0.5
IFSLosses = 0.30
QE = 0.68
PhCountingEff = 0.8
CTE = .893
offaxis.data*=IFSLosses*QE*PhCountingEff*CTE*pol
# offaxis.data*=0.68*0.8*0.893
print(np.sum(offaxis.data))
detectorFrame = propagateIFS(par,lamlist.value/1000.,offaxis)



In [ ]:
print(np.sum(offaxis.data)*0.11)
np.sum(offaxis.data)*0.11/3e-3

In [ ]:
rotated_list = glob.glob(par.exportDir+'/imagePlaneRot*nm.fits')
sum_rotated = np.zeros(Image(rotated_list[0]).data.shape)
for filename in rotated_list:
    sum_rotated += Image(filename).data
out = fits.HDUList(fits.PrimaryHDU(sum_rotated.astype(np.float32)))
out.writeto(par.exportDir + '/imagePlaneRot_sum.fits', clobber=True)


In [ ]:
print(np.sum(sum_rotated)*0.11)
print(np.sum(offaxis.data*0.11))


In [ ]:
import numpy as np
import glob
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
import sys
codefolder = '../../../../crispy'
if codefolder not in sys.path: sys.path.append(codefolder)
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.params import Params
codefolder = '../../../crispy'
par = Params(codefolder)
from astropy.io import fits as pyf
from crispy.IFS import polychromeIFS
from crispy.tools.image import Image

In [ ]:
inputCube = np.ones((1,512,512),dtype=float)
inCube = pyf.HDUList(pyf.PrimaryHDU(inputCube))
inCube[0].header['LAM_C'] = 770./1000.
inCube[0].header['PIXSIZE'] = 0.1
dlam=1.
par.saveRotatedInput=True

detectorFrame = polychromeIFS(par,[770.],inCube[0],dlambda=dlam,parallel=False)
filename = par.exportDir+'/det_660.fits'
Image(data=detectorFrame,header=par.hdr).write(filename)
